## Ch5 lab: cross-validation and the bootstrap

### 1. Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy # importing SciPu library to do scientific computing, e.g. numerical integration, optimisaiton, linear algebra
import pandas as pd 
import math
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from collections import OrderedDict # provides specialised container datatypes that are alternatives to built-in data structures like lists, tuples, dictionaries etc

# OrderedDict is a dictionary subclass that maintains the order of inserted items, useful for when the order of elements is important

### 2. The validation set approach

In [5]:
auto = pd.read_csv('data/Auto.csv', header = 0, na_values = '?')
auto = auto.dropna().reset_index(drop = True) # dropping the observations with NA values and reindex the observations from 0
print(auto.shape)
print(auto.head())

(392, 9)
    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0       130.0    3504          12.0    70   
1  15.0          8         350.0       165.0    3693          11.5    70   
2  18.0          8         318.0       150.0    3436          11.0    70   
3  16.0          8         304.0       150.0    3433          12.0    70   
4  17.0          8         302.0       140.0    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


In [8]:
# creating a training set
np.random.seed(1) # setting seed to ensure reproducibility in number generation
train = np.random.choice(auto.shape[0], 196, replace = False) # creating an array 'train' containing a random, non-repeated subset
select = np.in1d(range(auto.shape[0]), train) # creates a boolean arrray 'select' to show whether indicies in auto are in the train array or not

In [15]:
# building the model
lm = smf.ols ('mpg~horsepower', data = auto[select]).fit() # selecting a subset of the auto data 'select' so that we only receive our training data
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Mon, 15 May 2023   Prob (F-statistic):           1.28e-42
Time:                        09:48:01   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

In [11]:
# getting predictions for all observations in the dataset, using '~' to exclude results from the training samples
preds = lm.predict(auto)
square_error = (auto['mpg'] - preds)**2 # calculates squared error between actual targer values and predicted values
print('-------- Test error for 1st order model --------')
print(np.mean(square_error[~select])) # calculates the mean of the squared error for rows that... 
# do not satisfy the condition represented by '~select', i.e. rows that weren't in the training set

-------- Test error for 1st order model --------
23.361902892587235


In [14]:
# building a model with 2nd order of features
lm2 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0)', data = auto[select]).fit()

# getting predictions for all observations in the dataset, again using '~' to exclude training samples
preds = lm2.predict(auto)
square_error = (auto['mpg'] - preds)**2
print('--------Test error for 2nd order--------')
print(square_error[~select].mean()) # equivalent to above version, just different syntax

--------Test error for 2nd order--------
20.25269085835007


In [18]:
# building a model with 3rd order of features
lm3 = smf.ols('mpg~horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)', data = auto[select]).fit()

# getting predictions for all observations, excluding training samples
preds = lm3.predict(auto)
square_error = (auto['mpg'] - preds) ** 2 
print('--------Test error for 2nd order--------')
print(square_error[~select].mean()) # equivalent to above version, just different syntax

--------Test error for 2nd order--------
20.325609365773644


In [19]:
print(lm3.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     165.9
Date:                Mon, 15 May 2023   Prob (F-statistic):           4.60e-53
Time:                        09:50:12   Log-Likelihood:                -561.56
No. Observations:                 196   AIC:                             1131.
Df Residuals:                     192   BIC:                             1144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               66.5200 

### 3. Leave-one-out cross validation

Leave-one-out cross-validation (LOOCV) is a technique used to estimate the performance of a machine learning model. In LOOCV, the dataset is divided into multiple subsets, each consisting of <b>all the samples except one</b>. For each subset, a model is trained using the remaining samples and then evaluated on the sample that was left out. This process is repeated for each sample in the dataset, and the evaluation results are averaged to obtain an overall performance estimate.

In more detail, the LOOCV process is as follows:
- Starting with the original dataset, LOOCV iterates over each sample one by one, designating it as the validation or test sample.

- The remaining samples in the dataset are used as the training data to build a model. This ensures that the training model has access to as much data as possible, similar to training on the entire dataset.

- The model is then used to predict the outcome or target variable for the validation sample.

- The predicted value is compared to the actual value of the validation sample, and the performance metric of interest (e.g., accuracy, mean squared error, etc.) is computed.

- Steps 2-4 are repeated for each sample in the dataset, with each sample taking a turn as the validation sample.

- The performance metrics obtained for each validation sample are then averaged to obtain a single performance estimate for the model.

In [20]:
# OLS fit 
ols_fit = smf.ols ('mpg~horsepower', data = auto).fit()
print(ols_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [22]:
# GLM fit. Compare with OLS fit, the coeffs are the same
glm_fit = smf.glm('mpg~horsepower', data = auto).fit()
print(glm_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [38]:
# using libraries / modules already imported to perform kfold validation in Python, as a reminder they are:
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.pipeline import Pipeline

In [39]:
# retraining the model in sklearn
x = pd.DataFrame(auto.horsepower)
y = auto.mpg

model = LinearRegression()
model.fit(x, y)
print(model.intercept_)
print(model.coef_)

39.93586102117047
[-0.15784473]


In [40]:
# using kfold validation to evaluate the performance of the model
k_fold = KFold(n_splits=x.shape[0]) # initialised a KFold object 'k_fold'. 'n_splits' set to no. samples in input data 'x.shape[0]'
test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)

# applies cross-validation with following parameters:
# 'model' = object / estimator to be evaluated, as defined in code block above
# 'x' = input features / predictor variables
# 'y' = target variables / labels
# 'cv' = the cross-validation strategy or object, in this case 'f_fold' as defined on previous line
# 'scoring' = scoring metric used to evaluate model's performance, in this case negative mean squared error
# 'n_jobs' = no. CPU cores to use for computation, value of -1 indicates using all available CPU cores

print(np.mean(-test))

24.231513517929226


In [41]:
# using pipline tool for a higher order polynomial fit

A = OrderedDict()
n_split = x.shape[0] # assigns no. samples in input data 'x' to the variable 'n_split', represents no. splits / folds to be used in cross-validation
for porder in range(1, 21, 2): # range values from 1 to 21 with step of 2
    model = Pipeline([('poly', PolynomialFeatures(degree = porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits = n_split)
    test = cross_val_score(model, x, y, cv = k_fold, scoring = 'neg_mean_squared_error', n_jobs=-1)
    A[str(porder)] = np.mean(-test)

print(A)

OrderedDict([('1', 24.231513517929226), ('3', 19.334984064118647), ('5', 19.033206870017146), ('7', 19.125446846687602), ('9', 19.13392619499544), ('11', 19.136033100530025), ('13', 27.76341651771519), ('15', 35.29333367792178), ('17', 43.65451056724629), ('19', 60.96829353029924)])


#### Notes on interpretation

Overall, this code block performs polynomial regression with different polynomial orders, applies k-fold cross-validation to evaluate the performance of each model using negative mean squared error, and stores the mean squared error values in an OrderedDict. Finally, it prints the OrderedDict containing the mean squared error values for different polynomial orders.

- <code>model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])</code>: creates a pipeline for the model using the 'Pipeline' class from scikit-learn. The pipeline consists of two steps: 
    - Polynomial feature transformation using PolynomialFeatures with the specified degree porder
    - Linear regression using LinearRegression.

- <code>k_fold = KFold(n_splits=n_split) </code>: initializes a KFold object 'k_fold' for performing k-fold cross-validation. The 'n_splits' parameter is set to the number of samples in the input data ('n_split'), indicating that each sample will be used as the test set once in each fold.

- <code>test = cross_val_score(model, x, y, cv=k_fold, scoring='neg_mean_squared_error', n_jobs=-1)</code>: applies cross-validation to evaluate the performance of the pipeline model. The 'cross_val_score' function performs cross-validation by splitting the data into training and test sets according to the specified cross-validation strategy ('k_fold'), trains the pipeline model on each training set, evaluates the model's performance on the corresponding test set using negative mean squared error, and returns the scores for each fold. The negative mean squared error values for each fold are stored in the test variable.

- <code>A[str(porder)] = np.mean(-test)</code>: calculates the mean of the negative mean squared error values obtained from cross-validation ('-test') and stores it in the 'OrderedDict' 'A'. The polynomial order porder is used as the key in A to associate it with the corresponding mean squared error value.

In [37]:
# printing results from OrderedDict in a more readable format, one below another:

for key, value in A.items():
    print(f'{key}: {value}')

1: 24.231513517929226
3: 19.334984064118647
5: 19.033206870017146
7: 19.125446846687602
9: 19.13392619499544
11: 19.136033100530025
13: 27.76341651771519
15: 35.29333367792178
17: 43.65451056724629
19: 60.96829353029924


### 4. Boostrap sampling

Boostrap sampling involves creating multiple resamples of the dataset by randomly selecting observations from the original dataset with replacement. Each resample is of the same size as the original dataset, but some observations may be selected multiple times, while others may not be selected at all. This process allows for the creation of multiple datasets that are similar to the original dataset, but with slight variatinos due to the sampling process.

In [42]:
portfolio = pd.read_csv('data/Portfolio.csv', header = 0)

In [49]:
# defining a function 'alpha_fn' which calculates value called 'alpha' using a given dataset and an index of selected samples
def alpha_fn(data, index):
    X = data.X.iloc[index]
    Y = data.Y.iloc[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

# last line calculates the value of 'alpha', computing the numerator and denominator then dividing them to obtain final value

In [50]:
# calling the function 'alpha_fn' with two arguments, 'portfolio' and 'range'
alpha_fn(portfolio, range(0,100))

0.5766511516104116

#### Notes on interpretation for block above

By calling <code>alpha_fn(portfolio, range(0,100))</code>, the <code>alpha_fn</code> function is invoked with the portfolio dataset and a sequence of indices from 0 to 99. This means that the function will calculate the alpha value using the samples at indices 0 to 99 from the X and Y variables in the portfolio dataset.

In [51]:
# generates a random sample of 100 numbers chosen with replacement from 0 to 99
# then stores the numbers in ascending order using 'np.sort'
np.sort(np.random.choice(range(0,100), size = 100, replace = True))

array([ 0,  0,  1,  1,  1,  4,  5,  5,  7,  7,  7, 11, 11, 13, 14, 16, 16,
       17, 18, 21, 21, 21, 22, 24, 24, 25, 25, 26, 27, 27, 28, 31, 31, 32,
       32, 32, 32, 33, 33, 34, 34, 35, 36, 37, 38, 38, 38, 40, 40, 41, 42,
       43, 44, 45, 45, 45, 46, 48, 48, 50, 51, 52, 57, 58, 61, 62, 62, 64,
       66, 67, 67, 68, 68, 68, 71, 71, 72, 74, 75, 76, 76, 82, 82, 84, 85,
       88, 88, 88, 89, 92, 92, 94, 96, 96, 96, 97, 97, 97, 97, 99])

In [55]:
# calling the 'alpha_fn' again with two arguments, this time using the randomly generated range above
# results in alpha value calculates using random subset of samples from X and Y variables in the portfolio dataset
alpha_fn(portfolio, np.random.choice(range(0,100), size = 100, replace = True))

0.6301886002042987

In [60]:
# defining function to perform bootstrapping on a given dataset using a specified input function
# the function conducts bootstrapping for a specified no. iterations and returns the mean and std of the resulting statistic

def boot_python(data, input_fun, iteration):
    n = portfolio.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)
    for i in range(len(idx)):
        stat[i] = input_fun(data, idx[i]) 
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}
    
boot_python(portfolio, alpha_fn, 1000) # applying the function

{'Mean': 0.5815356030576532, 'STD': 0.09187477005226481}

#### Notes on interpretation

- <code>def boot_python(data, input_fun, iteration):</code>: This line defines the function boot_python that takes three arguments: data, which represents the dataset, input_fun, which is the input function used to calculate a statistic, and iteration, the number of bootstrap iterations.

- <code>n = Portfolio.shape[0]</code>: This line assumes there is a variable or object named Portfolio and assigns the number of rows in the Portfolio dataset to the variable n.

- <code>idx = np.random.randint(0, n, (iteration, n))</code>: This line generates a two-dimensional array idx using np.random.randint. Each row in idx contains a bootstrap sample of indices randomly selected from the range 0 to n-1 (the number of rows in the dataset). The iteration parameter determines the number of rows, and n specifies the range of random integers.

- <code>stat = np.zeros(iteration)</code>: This line initializes an array stat of size iteration filled with zeros. This array will store the statistics calculated by the input function for each bootstrap sample.

- <code>for i in range(len(idx)):</code>: This line starts a loop over the length of idx, iterating from 0 to len(idx)-1.

- <code>stat[i] = input_fun(data, idx[i])</code>: Within the loop, this line calls the input function input_fun with the data dataset and the i-th row of idx. It calculates a statistic using the specified input function and assigns the result to the i-th element of the stat array.

- <code>return {'Mean': np.mean(stat), 'STD': np.std(stat)}</code>: This line returns a dictionary containing the mean and standard deviation of the stat array. The mean and standard deviation are calculated using np.mean and np.std, respectively.